In [2]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
#load pcd file, filter, downsample
pcd = o3d.io.read_point_cloud("corridor_colored.pcd")

pcd.estimate_normals()
#pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd])


In [28]:
def detect_ground_plane(pcd):
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
    
    return plane_model, inliers
    
    

In [5]:
def compute_plane_transformation(plane1, plane2):
    """
    Computes the transformation between two plane equations of the form ax + by + cz + d = 0.

    Args:
        plane1 (tuple): Coefficients of the first plane equation (a1, b1, c1, d1).
        plane2 (tuple): Coefficients of the second plane equation (a2, b2, c2, d2).

    Returns:
        np.ndarray: A 4x4 transformation matrix.
    """
    # Extract coefficients of the first plane equation
    a1, b1, c1, d1 = plane1

    # Extract coefficients of the second plane equation
    a2, b2, c2, d2 = plane2

    # Compute the transformation matrix coefficients
    a = a2 * a1 + b2 * b1 + c2 *inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0]) c1
    b = a2 * d1 - a1 * d2
    c = b2 * d1 - b1 * d2
    d = c2 * d1 - c1 * d2

    transformation_matrix = np.array([[a, 0, 0, b],
                                      [0, a, 0, c],
                                      [0, 0, a, d],
                                      [0, 0, 0, 1]])

    return transformation_matrix

In [29]:
ground_plane = np.asarray([0,0,1,0])
pcd_ground_plane, inliers = detect_ground_plane(pcd)
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

trans = compute_plane_transformation( pcd_ground_plane, ground_plane)

pcd_t = cp.deepcopy(pcd).transform(trans)

o3d.visualization.draw_geometries([pcd, inlier_cloud])

In [7]:
o3d.io.write_point_cloud("corridor_ground_align.pcd", pcd_t)

True

In [40]:
bb = pcd_t.get_axis_aligned_bounding_box()
min_bound = bb.get_min_bound()
max_bound = bb.get_max_bound()

min_bound[0] = -47
min_bound[1] = -3
min_bound[2] = - 2

max_bound[0] = 3
#max_bound[1] =
max_bound[2] = 3
print(min_bound, max_bound)

pcd_new = cp.deepcopy(pcd_t)
crob_bb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
pcd_new = pcd_new.crop(crob_bb)


o3d.visualization.draw_geometries([pcd_new])

[-47.  -3.  -2.] [3.         3.49200176 3.        ]


In [41]:
o3d.io.write_point_cloud("corridor_ground_align.pcd", pcd_new)

True